In [1]:
import os 
os.chdir('/Users/marcs')

In [2]:
import pandas as pd
import pickle
from pickle_loader import pickle_loader
import numpy as np

In [7]:
final_fundamental_dict = pickle_loader('/Users/marcs/OneDrive/Documents/stock_analysis2/final_fundamental_dict.pickle')
modeling_attributes = pickle_loader("/Users/marcs/OneDrive/Documents/stock_analysis2/fundamental_modeling_attributes.pickle")

In [13]:
modeling_attributes

{'Accounts Receivable',
 'Basic Average Shares',
 'Basic EPS',
 'Beginning Cash Position',
 'Capital Expenditure',
 'Capital Stock',
 'Cash And Cash Equivalents',
 'Cash Cash Equivalents And Short Term Investments',
 'Cash Flow From Continuing Financing Activities',
 'Cash Flow From Continuing Investing Activities',
 'Cash Flow From Continuing Operating Activities',
 'Change In Working Capital',
 'Changes In Cash',
 'Common Stock',
 'Common Stock Equity',
 'Current Assets',
 'Current Liabilities',
 'Diluted Average Shares',
 'Diluted EPS',
 'Diluted NI Availto Com Stockholders',
 'EBIT',
 'End Cash Position',
 'Financing Cash Flow',
 'Free Cash Flow',
 'Gains Losses Not Affecting Retained Earnings',
 'Invested Capital',
 'Investing Cash Flow',
 'Long Term Debt',
 'Long Term Debt And Capital Lease Obligation',
 'Net Income',
 'Net Income Common Stockholders',
 'Net Income Continuous Operations',
 'Net Income From Continuing And Discontinued Operation',
 'Net Income From Continuing Opera

In [8]:
def percentage_diff(arr):
    x1 = arr[1:]
    x0 = arr[:-1]
    
    np.seterr(divide='ignore', invalid='ignore')
    diff = ((x1 - x0) / x0) * 100 
    return diff

In [9]:
def get_non_impute_data(final_fundamental_dict, modeling_attributes):
    
    tickers = list(final_fundamental_dict.keys())
    final_dataset = {}
    
    for ticker in tickers:
        data = final_fundamental_dict[ticker]
        ticker_attr = set(data.index)
        final_ticker_dataset = np.zeros((len(modeling_attributes), 3))
        
        for i, attr in enumerate(modeling_attributes):
            if attr in ticker_attr:
                attr_array = np.array(list(data.loc[attr]))
                #print(ticker, attr, attr_array)
                if len(attr_array) > 4:
                    attr_array = attr_array[:-1]
                
                attr_array = percentage_diff(attr_array)
                final_ticker_dataset[i, :] = attr_array
            else:
                attr_array = np.full(3, np.nan)
                final_ticker_dataset[i, :] = attr_array
            
        final_ticker_df = pd.DataFrame(final_ticker_dataset, columns = ['Q1', 'Q2', 'Q3'], index = list(modeling_attributes))
        final_dataset[ticker] = final_ticker_df
    
    return final_dataset

In [10]:
final_dataset_non_impute = get_non_impute_data(final_fundamental_dict, modeling_attributes)

In [16]:
final_dataset_non_impute['ACN'].index

Index(['Basic Average Shares', 'EBIT', 'Invested Capital',
       'Diluted Average Shares', 'Diluted NI Availto Com Stockholders',
       'Cash Cash Equivalents And Short Term Investments',
       'Cash And Cash Equivalents', 'Financing Cash Flow', 'Capital Stock',
       'Net Interest Income', 'Stockholders Equity', 'Free Cash Flow',
       'Normalized Income', 'Cash Flow From Continuing Financing Activities',
       'Net Income', 'Ordinary Shares Number', 'Operating Cash Flow',
       'Cash Flow From Continuing Operating Activities', 'Common Stock',
       'Net Income Continuous Operations', 'Net Tangible Assets',
       'Total Non Current Assets', 'Capital Expenditure',
       'Total Liabilities Net Minority Interest', 'Long Term Debt',
       'Tangible Book Value', 'Share Issued', 'Investing Cash Flow',
       'Change In Working Capital', 'Total Debt', 'Basic EPS', 'Tax Provision',
       'End Cash Position', 'Tax Effect Of Unusual Items',
       'Gains Losses Not Affecting Retaine

In [12]:
with open("/Users/marcs/OneDrive/Documents/stock_analysis2/final_dataset_non_impute.pickle", 'wb') as f:
        pickle.dump(final_dataset_non_impute, f)